# Extracting Balanced Positive

In [1]:
#Imports
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from collections import Counter
import pandas as pd
import json

In [2]:
#Loading the Dataset
dataset = pd.read_csv('/Users/sankeerthana/Documents/NTU/YEAR_4/SEM_2/CZ4034/IR-Sentiment-Analysis/IR-Sentiment-Analysis/final_codes/RNN/clean_processed_flair_labels.csv')
dataset.drop('Unnamed: 0', axis=1, inplace=True)

# # of rows = 5000, # of cols = 2
print(dataset.shape)

#Exploring the Dataset
print(dataset['flair_labels'].value_counts())

(5000, 2)
1    4020
0     980
Name: flair_labels, dtype: int64


In [3]:
positive_df = dataset[dataset['flair_labels'] == 1]
positive_df.reset_index(inplace=True)
positive_df.drop('index', axis=1, inplace=True)
positive_df

/var/folders/54/f2chvxk17fd0vhl03xl62d_80000gn/T/ipykernel_1590/787625808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df.drop('index', axis=1, inplace=True)


,Review,flair_labels
0,oh wanted buy acwell licorice toner stock ende...,1
1,stop using putting product face month two acne...,1
2,looking way fade sunspots sides face also want...,1
3,feels lightweight smells good love feels light...,1
4,one top picks traveling especially using long ...,1
...,...,...
4015,used two weeks notice effect complexion looks ...,1
4016,love product much texture smooth light makes s...,1
4017,worked wonders skin skin soft glowing break ca...,1
4018,never liked use serum always break decided giv...,1


In [4]:
negative_df = dataset[dataset['flair_labels'] == 0]
negative_df.reset_index(inplace=True)
negative_df.drop('index', axis=1, inplace=True)
negative_df

/var/folders/54/f2chvxk17fd0vhl03xl62d_80000gn/T/ipykernel_1590/3441776143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df.drop('index', axis=1, inplace=True)


,Review,flair_labels
0,despite keeping lip cap sealed like youre inst...,0
1,im love toner doesnt seem anything special get...,0
2,love klairs excited try moisturizer read mixed...,0
3,see major change others saying make skin dewy ...,0
4,didnt like product cleansed well except face f...,0
...,...,...
975,purchased mask help treat pimples around mouth...,0
976,feel nice put absorbs nicely didnt see results...,0
977,wanted like lot work even rinsing shower would...,0
978,sensitive skin products irritate redden skin e...,0


In [5]:
subset_positive = positive_df.sample(n=980) #cuz len(negative_df) = 980
subset_positive.reset_index(inplace=True)
subset_positive.drop('index', axis=1, inplace=True)
subset_positive

,Review,flair_labels
0,havent found acne patches better gave four sta...,1
1,im love feel see immediate difference skin tex...,1
2,think good moisturizer little goes long way so...,1
3,ive tried neogen day light protection sunscree...,1
4,acneprone cranky skin soaks thick takes time a...,1
...,...,...
975,first saw reviews skin drinks didnt understand...,1
976,great exfoilater makes skin feel clean stripped,1
977,using weeks cant say seen much improvement bit...,1
978,lot cleansers dry onemy skin always feels supe...,1


In [6]:
balanced_dataset = pd.concat([subset_positive, negative_df])
balanced_dataset

,Review,flair_labels
0,havent found acne patches better gave four sta...,1
1,im love feel see immediate difference skin tex...,1
2,think good moisturizer little goes long way so...,1
3,ive tried neogen day light protection sunscree...,1
4,acneprone cranky skin soaks thick takes time a...,1
...,...,...
975,purchased mask help treat pimples around mouth...,0
976,feel nice put absorbs nicely didnt see results...,0
977,wanted like lot work even rinsing shower would...,0
978,sensitive skin products irritate redden skin e...,0


In [7]:
from sklearn.utils import shuffle
balanced_dataset = shuffle(balanced_dataset)
balanced_dataset.reset_index(inplace=True)
balanced_dataset

,index,Review,flair_labels
0,844,best impulse buy ive made sokoglam im pleased ...,1
1,558,love serum great consistency absorbs well wake...,1
2,447,warm oily skin doesnt absorb rapidly prefer on...,0
3,680,cute work really well 1010,1
4,664,used week half see changes skin feels soft,0
...,...,...,...
1955,911,made skin soooo red irritated burned thought g...,0
1956,929,ive using cleanser several weeks noticed diffe...,0
1957,306,im halfway first bottle love product absorbs w...,1
1958,511,sensitive skin thats prone redness ive using d...,1


In [8]:
balanced_dataset.drop('index', axis=1, inplace=True)
balanced_dataset

,Review,flair_labels
0,best impulse buy ive made sokoglam im pleased ...,1
1,love serum great consistency absorbs well wake...,1
2,warm oily skin doesnt absorb rapidly prefer on...,0
3,cute work really well 1010,1
4,used week half see changes skin feels soft,0
...,...,...
1955,made skin soooo red irritated burned thought g...,0
1956,ive using cleanser several weeks noticed diffe...,0
1957,im halfway first bottle love product absorbs w...,1
1958,sensitive skin thats prone redness ive using d...,1


In [9]:
balanced_dataset.to_csv('/Users/sankeerthana/Documents/NTU/YEAR_4/SEM_2/CZ4034/IR-Sentiment-Analysis/IR-Sentiment-Analysis/final_codes/BERT/balanced_data.csv')